In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
spark = SparkSession.builder.appName("ETL").getOrCreate()

## Extract Data

In [0]:
compressed_file_path = "staging/equipment_failure_sensors.txt.gz"
equipament_sensors_path = "staging/equipment_sensors.csv"
equipament_path = "staging/equipment.json"

In [0]:
schema = StructType(
    [
        StructField("equipment_id", IntegerType(), True),
        StructField("sensor_id", IntegerType(), True)
    ]
    )

equipment_failure_sensors_df = spark.read.text(compressed_file_path)
equipament_sensors_df = spark.read.schema(schema).csv(equipament_sensors_path, header=True, sep=",")
equipament_df = spark.read.json(equipament_path, multiLine=True)

equipment_failure_sensors_df.show(5, truncate=False)
equipament_sensors_df.show(5, truncate=False)
equipament_df.show(5, truncate=False)

In [0]:
equipment_failure_sensors_df = equipment_failure_sensors_df.withColumn(
    "value", regexp_replace("value", r"(\d{4})/(\d{2})/(\d{1,2})", "$1-$2-$3 00:00:00"))


equipment_failure_sensors_df = equipment_failure_sensors_df.select(
    regexp_extract("value", r"^\[(.+)\]\t", 1).alias("timestamp"),
    regexp_extract("value", r"\]\t(\w+)\t", 1).alias("level"),
    regexp_extract("value", r"sensor\[(\d+)\]", 1).alias("sensor_id"),
    regexp_extract("value", r"temperature\t(-?[\d\.]+)", 1).alias("temperature"),
    regexp_extract("value", r"vibration\t(-?[\d\.]+)", 1).alias("vibration"),
)



equipment_failure_sensors_df.show(5, truncate=False)

In [0]:
equipment_failure_sensors_df = equipment_failure_sensors_df.withColumn("level", col("level").cast(StringType())) \
    .withColumn("timestamp", col("timestamp").cast(TimestampType())) \
    .withColumn("sensor_id", col("sensor_id").cast(IntegerType())) \
    .withColumn("temperature", col("temperature").cast(DoubleType())) \
    .withColumn("vibration", col("vibration").cast(DoubleType()))

equipment_failure_sensors_df.printSchema()

In [0]:
equipament_sensors_df = equipament_sensors_df.withColumn("equipment_id", col("equipment_id").cast(IntegerType())) \
    .withColumn("sensor_id", col("sensor_id").cast(IntegerType()))

equipament_sensors_df.printSchema()

In [0]:
equipament_df = equipament_df.withColumn("equipment_id", col("equipment_id").cast(IntegerType())) \
    .withColumn("group_name", col("group_name").cast(StringType())) \
    .withColumn("name", col("name").cast(StringType()))

equipament_df.printSchema()

In [0]:
equipment_failure_sensors_df.write.mode("overwrite").parquet("raw/equipment_failure_sensors.parquet")
equipament_sensors_df.write.mode("overwrite").parquet("raw/equipament_sensors.parquet")
equipament_df.write.mode("overwrite").parquet("raw/equipament.parquet")

## Star Schema

In [0]:
equipment_failure_sensors_df = spark.read.parquet("raw/equipment_failure_sensors.parquet")
equipament_sensors_df = spark.read.parquet("raw/equipament_sensors.parquet")
equipament_df = spark.read.parquet("raw/equipament.parquet")


In [0]:
# Join three tables
fact_table = equipment_failure_sensors_df.join(equipament_sensors_df, "sensor_id", "left") \
    .select("sensor_id", "equipment_id", "timestamp", "level", "temperature", "vibration")

fact_table.show(5, truncate=False)

fact_table.write.mode("overwrite").partitionBy("equipment_id", "sensor_id").parquet("prepared/equipment_failure.parquet")

In [0]:
equipament_dim = equipament_df.select("equipment_id", "group_name", "name")
equipament_dim.show(5, truncate=False)
equipament_dim.write.mode("overwrite").parquet("prepared/equipament.parquet")

## Analytics

In [0]:
equipment_failure_sensors_df.groupBy("level").count().show()

In [0]:
print("Total equipment failures: ", equipment_failure_sensors_df.count())

In [0]:
# Join equipment_failure_sensors with equipament_sensors
df = equipment_failure_sensors_df.join(equipament_sensors_df, equipment_failure_sensors_df.sensor == equipament_sensors_df.sensor_id, "left")\
    .drop("sensor_id", "sensor", "temperature", "vibration")

df.show(5, truncate=False)

In [0]:
df.groupBy("equipment_id").count().show(5, truncate=False)